In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import os, re, magic, json, sys
import time, urllib
from googlesearch import search
from datetime import datetime, timedelta
from pymongo import MongoClient
from search_engine_scraper import google_search,bing_search,yahoo_search
import requests
from bs4 import BeautifulSoup
import logging, logging.handlers
import random
import subprocess
import signal, unicodedata
import html2text
from contextlib import contextmanager
from bs4 import BeautifulSoup, UnicodeDammit
import pprint
import PyPDF2
from time import mktime, strptime
from datetime import datetime
import json
import spacy
from ipynb.fs.full.similarity_measure import compute_best_doc
from nltk.corpus import wordnet

The time recorded in the file is :  1617634135.1432438
The time now  is :  1617634947.0794308
A total difference of 13.532269783814748 minutes
It has been less than 15 minutes since the proxies were renewed, therefore sticking with the old proxies


In [29]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#def constant():
CHEMIN_LOG = "./log/"
CHEMIN_RESULTATS = "./documents/"
NOT_SITES = "-site:youtube.com -site:pagesjaunes.fr"


In [30]:
class Log(object):
    def __init__(self, dossier, nomFichier, niveau=logging.DEBUG):
        super(Log, self).__init__()

        self.__logger__ = logging.getLogger(nomFichier)
        self.__logger__.setLevel(niveau)

        format = logging.Formatter('%(asctime)s :: %(levelname)s :: %(message)s', datefmt='%d-%m-%Y %H:%M:%S')
        fichierLog = logging.handlers.RotatingFileHandler("{0}{1}.log".format(dossier, nomFichier), 'a', 1000000, 1)

        fichierLog.setLevel(niveau)
        fichierLog.setFormatter(format)
        self.__logger__.addHandler(fichierLog)

        console = logging.StreamHandler()
        console.setLevel(niveau)
        self.__logger__.addHandler(console)

    def info(self, message):
        self.__logger__.info(message)

    def debug(self, message):
        self.__logger__.debug(message)

    def warning(self, message):
        self.__logger__.warning(message)

    def error(self, message):
        self.__logger__.error(message)

    def critical(self, message):
        self.__logger__.critical(message)

    def exception(self, message):
        self.__logger__.exception(message)

    def close(self):
        for handler in  self.__logger__.handlers[:] :
            handler.close()
            self.__logger__.removeHandler(handler)


In [31]:
def pause(logger, minutes=0.5):
    """
		Effectue une pause.

		@type  log: Logger.
		@param log: Fichier de log.

		@type  minutes: Entier.
		@param minutes: Temps en minute à attendre.
	"""
    date = datetime.now().strftime('%d-%m-%Y:%Hh%M')
    current_time = datetime.now() + timedelta(minutes=minutes)

    logger.info("{0} : Nombre limite de requete atteint. Reprise du programme : {1}".format(date, current_time.strftime(
        '%d-%m-%Y:%Hh%M')))

    while datetime.now() < current_time:
        time.sleep(0.5)

In [32]:
def generation_requetes_(ville, motscles, logger, site):
    
    logger.info("Génération des requêtes")

    requetes_effectuees = []

    with open("{0}{1}/.sauvegarde.txt".format(CHEMIN_RESULTATS, ville)) as fichier:
        requetes_effectuees = fichier.readlines()

    for i in range(0, len(motscles), 1):
        # motscles_couple = motscles[i].split("+")

        # Si aucun site n'est spécifié alors on n'exclut
        # seulement ceux définis dans les constants
        if site == "":
            site_or_not_sites = NOT_SITES
        # Sinon on cherche uniquement sur le site spécifié
        else:
            site_or_not_sites = "site:" + site
        requete = "\"{0}\" AND {1}".format(ville,motscles[i])
        if not any(requete in s for s in requetes_effectuees):
            yield requete

In [33]:
def insertion_document(document, collection):
    """ Insère un document (dictionnaire) dans une collection MongoDB """
    resultat = collection.insert_one(document)
    return resultat.inserted_id


In [34]:
def format_to_iso_date(pdfdate): 
    datestring = ''
    if pdfdate !='':
        if len(pdfdate)==23:
            datestring = pdfdate[2:-7]
        elif len(pdfdate)==17:
            datestring = pdfdate[2:-1] 
        elif len(pdfdate)==21:
            datestring = pdfdate[:-7]
        ts = strptime(datestring,"%Y%m%d%H%M%S")# "%Y-%m-%dT%H:%M:%S.%fZ" )# 
        #print(ts)
        dt = datetime.fromtimestamp(mktime(ts))
        #print(dt)
        #new_format = dt.strftime("%Y-%m-%dT%H:%M:%S.%fZ")# ('%m-%d-%Y')
        new_format = dt.isoformat()
    else:
        new_format = "no_date"
    return new_format

In [35]:
'''def get_pdf_info(f):
    #pp = pprint.PrettyPrinter(indent=4)
    fd = PyPDF2.PdfFileReader(f, 'rb')
    doc_info = fd.getDocumentInfo()
    #pp.pprint(doc_info)
    return format_to_iso_date(doc_info['/ModDate'])'''

"def get_pdf_info(f):\n    #pp = pprint.PrettyPrinter(indent=4)\n    fd = PyPDF2.PdfFileReader(f, 'rb')\n    doc_info = fd.getDocumentInfo()\n    #pp.pprint(doc_info)\n    return format_to_iso_date(doc_info['/ModDate'])"

In [36]:
def get_pdf_info(f):
    #pp = pprint.PrettyPrinter(indent=4)
    fd = PyPDF2.PdfFileReader(f, 'rb')
    doc_info = fd.getDocumentInfo()
    if '/ModDate' in doc_info:
        val_ = doc_info['/ModDate']
    elif '/CreationDate' in doc_info:
        val_ = doc_info['/CreationDate']
    else:
        pass

    return format_to_iso_date(val_)

In [37]:
# extract spatial named entities
def SNE_Extract(title_mtd):
    SNE = {}
    nlp = spacy.load('fr_core_news_sm')# Text with nlp
    dc = nlp(title_mtd)
    i = 0
    for ent in dc.ents:
        if ent.label_ in ['LOC']:
            SNE['ent'+repr(i)] = repr(ent)
            i+=1
    return SNE

In [38]:
# cette fonction ne prend que la date de publis
#à adapter pour prendre en compte l'extraction auto d'entitées nommées temporelles dans les titres
def TNE_extract(mtd): #ajouter title_mtd pour la prise en compte des ENT presentent dans le titles
    mtd['TNE'] = {}
    if 'post_date' in mtd and '$date' in mtd['post_date']:        
        mtd['TNE']['date'] = mtd['post_date']['$date']
    return mtd

In [39]:
# ajout de meta données suplementaires pour l'intdexation spatiale et temporelle
def enrich_mtd(mtd):
    if "title" in mtd:
        title_mtd = mtd['title']
        SNE = SNE_Extract(title_mtd)
        mtd['SNE'] = SNE #liste d'ENS
        
    mtd['TNE'] = {}
    #if 'post_date' in mtd and '$date' in mtd['post_date']:        
    #    mtd['TNE']['date'] = mtd['post_date']['$date']
    if "post_date" in mtd: 
        mtd['TNE']['date'] = []
        mtd['TNE']['date'] = mtd['post_date']
    #mtd = TNE_extract(mtd)
    return mtd

In [40]:
# quelques étapes de preprocess
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
class TimeoutException(Exception): pass

@contextmanager
def limitation_temporelle(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException

    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)

    try:
        yield
    finally:
        signal.alarm(0)

def norm_string(string):
	""" On supprime les accents et on remplace les caractères spéciaux par un tiret bas """
	new_string = unicodedata.normalize('NFKD', string).encode('ascii', 'ignore').decode('utf-8')
	return re.sub(r'[^\w\[\]\./]+', '_', new_string)

def formatage_mots(mot):
    """
        Supprime les retour charriot et remplace les espace par un ?.

        @type  mot: String.
        @param mot: Mot à formaté.

        @rtype: String.
        @return: Mot formaté.
    """
    liste_caracteres = dict()

    liste_caracteres["\n"] = ""
    liste_caracteres[" "] = "?"

    for i in liste_caracteres :
        mot = mot.replace(i, liste_caracteres[i])

    return mot

def get_paire(ligne):
    """
        Extrait du texte la clé et la valeur.

        @type  ligne: String.
        @param ligne: Texte contenant les valeurs.

        @rtype: Tuple.
        @return: Valeurs extraites.
    """
    cle, sep, valeur = ligne.strip().partition("=")
    return cle.replace(" ",""), valeur.replace(" ","")

def creation_dossier_resultat(chemin_resultats, ville, log, keyword_file) :
    """
        Création des dossiers de stockage et renvoi des mots clés.

        @type  ville: String.
        @param ville: Nom de la ville.

        @rtype: Liste de string.
        @return: Liste des mots clés à rechercher.
    """
    if not os.path.exists(chemin_resultats + ville):
        os.makedirs(chemin_resultats + ville)
        log.info("Création des dossier pour {0}".format(ville))
    if not os.path.exists("{0}{1}/Documents_SRC".format(chemin_resultats, ville)):
        os.makedirs("{0}{1}/Documents_SRC".format(chemin_resultats, ville))
        log.info("Création du dossier Documents_SRC\n")

    with open(keyword_file) as keywords_file:
        words = keywords_file.readlines()
        log.info("Lecture de keywords.txt ... \n")

    open("{0}{1}/.sauvegarde.txt".format(chemin_resultats, ville),"a").close()
    for i in range(0, len(words)):
        words[i] = formatage_mots(words[i])
    sampling = random.choices(words, k=3)
    #print('>>',words)
    #print('>>>',sampling)

    return words


def extract_date(text):
    # Motif : JJ/MM/AAAA
    match_date = re.search(r'(\d{2})/(\d{2})/(\d{4})', text)
    # Motif : JJ mois AAAA
    match_date2 = re.search(r'(\d{1,2}) (\w+) (\d{4})', text)

    if match_date:
        date = datetime(
            int(match_date[3]), # Année
            int(match_date[2]), # Mois
            int(match_date[1])  # Jour
        )

        return date

    elif match_date2:
        mapping_month = {
            'janvier': 1,
            'février': 2,
            'mars': 3,
            'avril': 4,
            'mai': 5,
            'juin': 6,
            'juillet': 7,
            'août': 8,
            'septembre': 9,
            'octobre': 10,
            'novembre': 11,
            'décembre': 12,
        }

        month = match_date2[2].lower()

        if month in mapping_month:
            date = datetime(
                int(match_date2[3]), # Année
                mapping_month[month], # Mois
                int(match_date2[1])  # Jour
            )

            return date

def html_date_to_isoformat(date):
    #x = "2020-09-18T00:00:00.000Z"
    ts = strptime(date,"%Y-%m-%dT%H:%M:%S.%fZ")
    dt = datetime.fromtimestamp(mktime(ts))
    return dt.isoformat()

def less_html(html_doc):
    """ Prend du code HTML en entrée et retourne un code épuré de certaines balises """
    soup = BeautifulSoup(html_doc, 'html.parser')
    # Regex pour matcher les attributs contenant ces termes
    bad_classes = re.compile(r'menu|head|publici|share|social|button|alert|prev|next|foot|tags|label|sidebar|author|topics|contact|modal|nav|snippet|register|aside|logo|bandeau|immobilier', re.IGNORECASE)
    # Suppression des espaces ou des sauts de ligne au début et à la fin du titre
    title = re.sub(r'^\s|\s$', '', soup.find('title').text)

    # Dictionnaire des métadonnées
    metadata = {}
    metadata['title'] = title
    # Recherche d'une éventuelle balise dont la classe contient "date"
    # En principe la première date est la date de publication
    bloc_date = soup.find(class_=re.compile(r'date', re.IGNORECASE))

    if bloc_date:
        # Recherche du premier motif JJ/MM/AAAA,
        # en principe la date de publication
        metadata['post_date'] = ''
        date = extract_date(bloc_date.text)
        if date: metadata['post_date'] = html_date_to_isoformat(date)

    for balise in soup.find_all():
        conditions = (
            balise.name == 'head',
            balise.name == 'nav',
            balise.name == 'footer',
            balise.name == 'aside',
            balise.name == 'script',
            balise.name == 'style',
            balise.name == 'a',
            balise.name == 'figure',
            balise.name == 'img',
            balise.name == 'svg',
            balise.name == 'noscript',
            balise.name == 'form',
            balise.name == 'button'
        )

        if any(conditions):
            balise.extract()
        # On ajoute un espace devant chaque span, pour éviter
        # parfois d'avoir des mots collés
        elif balise.name == 'span' and balise.string:
            balise.string = ' ' + balise.string

    for balise in soup.find_all(attrs={'class': bad_classes}):
        balise.decompose()

    for balise in soup.find_all(attrs={'id': bad_classes}):
        balise.decompose()

    for balise in soup.find_all():
        if balise.text == '': balise.extract()

    return metadata, str(soup)

def convert_pdf_to_txt(src_file_path):
    """
        Appel externe à pdftotext.

        -q : pas de message d'erreur dans la sortie.
         - : envoie la sortie dans la console au lieu d'un fichier texte.

        Capture de la sortie texte.

        @type  src_file_path: String.
        @param src_file_path: Chemin du fichier source.

        @rtype: String.
        @return: Texte brut.
    """
    completed_process = subprocess.run(["pdftotext", "-q", src_file_path, "-"], stdout=subprocess.PIPE)
    return completed_process.stdout.decode('utf-8')


def convert_html_to_txt(src_file_path):
    """
        Conversion à l'aide de html2text.
        Détection automatique de l'encodage (UnicodeDammit).
        On capture la sortie texte.

        @type  src_file_path: String.
        @param src_file_path: Chemin du fichier source.

        @rtype: String.
        @return: Texte brut.
    """
    html_file = open(src_file_path, 'rb').read()
    dammit = UnicodeDammit(html_file)
    metadata, html_mini = less_html(html_file.decode(dammit.original_encoding))

    handler = html2text.HTML2Text()
    handler.ignore_links = True
    handler.ignore_emphasis = True
    text = handler.handle(html_mini)

    return metadata, text

In [41]:
class TextCleaner():
    """docstring for TextCleaner."""
    def __init__(self):
        """ Expressions régulières """
        self.match_alpha = re.compile(r'\w{3,}') # Existence d'un mot d'au moins 3 lettres
        self.match_formfeed = re.compile(r'\f') # Saut de page
        self.match_useless_line = re.compile(r'^[^\w•]+$|^[\d\. ]+$', re.MULTILINE) # Ligne ne contenant aucun caractère alphanumérique

        self.match_bad_nl = re.compile(r'([^\.?!\n:])\n+(?![IVX\d]+[\.\)]|ANNEXE)([\w\(«\"=<>])') # Mauvais saut de ligne
        self.match_bad_nl2 = re.compile(r'(\.{4,} ?\d+) (\w)') # Mauvais saut de ligne
        self.make_paragraph = re.compile(r'\.\n(?=\w)') # On sépare mieux les paragraphes
        self.match_toomuch_nl = re.compile(r'\n{3,}') # Nouvelles lignes surnuméraires
        self.match_begin_end_nl = re.compile(r'^\n+|\n+$') # Nouvelles lignes au début et à la fin

        self.match_begin_end_space = re.compile(r'^[ \t]+|[ \t]+$', re.MULTILINE) # Espace ou tabulation en début et fin de ligne
        self.match_toomuch_spaces = re.compile(r' {2,}|\t+') # Espaces surnuméraires et tabulations

        self.match_link = re.compile(r'!?\[.*?\]\(.*?\)|https?://[^ ]+', re.DOTALL) # Lien issu de la conversion depuis HTML
        self.match_cesure = re.compile(r'(\w)-(\w)') # Césure
        self.match_stuckwords = re.compile(r'(/\d{2,4}|[a-z])([A-ZÉÈÀÔ])') # Dates et mots collés

        self.match_odd = re.compile(r'[�●§\\\|]+|\(cid:\d+\)|(?<=- )W ') # caractère bizarre
        self.match_accent1 = re.compile(r'Ã©') # é
        self.match_accent2 = re.compile(r'Ã¨') # è
        self.match_accent3 = re.compile(r'Ã') # à
        self.match_puce1 = re.compile(r'')
        self.match_puce2 = re.compile(r'[]')
        self.match_diam = re.compile(r'diam\.')
        self.match_apostrophe = re.compile(r'’')

    def clean(self, text):
        # Remplacement des espaces insécables
        text = text.replace(u'\xa0', ' ')

        text = self.match_link.sub('', text)
        text = self.match_formfeed.sub('\n\n', text)
        text = self.match_useless_line.sub('\n', text)

        text = self.match_diam.sub('diamètre', text)
        text = self.match_accent1.sub('é', text)
        text = self.match_accent2.sub('è', text)
        text = self.match_accent3.sub('à', text)
        text = self.match_puce1.sub('*', text)
        text = self.match_puce2.sub('-', text)
        text = self.match_odd.sub('', text)
        text = self.match_apostrophe.sub('\'', text)

        text = self.match_begin_end_space.sub('', text)
        text = self.match_bad_nl.sub(r'\g<1> \g<2>', text)
        # On double la réparation des lignes, meilleurs résultats
        text = self.match_bad_nl.sub(r'\g<1> \g<2>', text)
        text = self.match_bad_nl2.sub(r'\g<1>\n\g<2>', text)
        text = self.make_paragraph.sub('.\n\n', text)
        text = self.match_stuckwords.sub(r'\g<1> \g<2>', text)
        text = self.match_toomuch_spaces.sub(' ', text)
        text = self.match_toomuch_nl.sub('\n\n', text)
        text = self.match_begin_end_nl.sub('', text)

        return text

    def exists_alpha(self, text):
        """ Contrôle l'existence de caractères alphanumériques """
        return self.match_alpha.search(text) is not None


In [42]:
def sauvegarde_fichier_simpl(ville, url, logger, tc):
    """
		Enregistre un fichier quelqu'il soit.

		@type  ville: String.
		@param ville: Nom de la ville.

		@type  url: String.
		@param url: URL vers le fichier pdf.

		@type  log: Logger.
		@param log: Fichier de log.

		@type  tc: TextCleaner.
		@param tc: Nettoyeur de texte.
	"""
    # On prend l'URL de base pour désigner l'origine du fichier à laquelle on enlève le "www."
    origine = re.sub(r'^www\.', '', urllib.request.urlparse(url).netloc)
    url_split = url.split('/')

    # Extraction du nom du document
    if url_split[-1] == '':
        nom_document = norm_string(url_split[-2])
    else:
        nom_document = norm_string(url_split[-1])

    src_file_path = "{0}{1}/Documents_SRC/[{2}]{3}".format(CHEMIN_RESULTATS, ville, origine, nom_document)

    if not os.path.isfile(src_file_path):
        req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
        response = urllib.request.urlopen(req)
        read_buffer = response.read()
        mime_type = magic.from_buffer(read_buffer, mime=True)

        source = {
            'file_link': src_file_path,
            'complete_url': url,
            'base_url': origine,
            'open_access': False,
        }

        document = {
            'name': nom_document,
            'mime_type': mime_type,
            'source': source,
            'manually_validated': False,
        }

        # Écriture du fichier téléchargé
        with open(src_file_path, "wb") as fichier:
            fichier.write(read_buffer)
            logger.info("Document sauvegardé.")

        # On détecte le type du document source pour utiliser la conversion adaptée
        # Si le document est un PDF
        if mime_type == 'application/pdf':
            try:
                # Conversion en texte brut
                raw_text = convert_pdf_to_txt(src_file_path)
                logger.info("Document PDF converti en texte brut.")
                p_d = get_pdf_info(src_file_path)
                document['text'] = tc.clean(raw_text)
                document['post_date'] = p_d
                logger.info("Texte nettoyé.\n")
            except:
                logger.exception("Erreur lors de la conversion en texte du pdf.\n")

        # Si le document est un HTML (ou autre fichier web)
        elif mime_type == 'text/html':
            try:
                # Conversion en texte brut
                metadata, raw_text = convert_html_to_txt(src_file_path)
                logger.info("Document web converti en texte brut.")
                document['text'] = tc.clean(raw_text)
                document.update(metadata)
                logger.info("Texte nettoyé.\n")

            except:
                logger.exception("Erreur lors de la conversion en texte de la page web.\n")

        else:
            logger.info("Autre type de document téléchargé.\n")
        
        #document = enrich_mtd(document)

        return document

    else:
        logger.info("Le document existe déjà.\n")
        return None

In [43]:
def sauvegarde_fichier_advanced(ville, url, logger, tc):
    """
		Enregistre un fichier quelqu'il soit.

		@type  ville: String.
		@param ville: Nom de la ville.

		@type  url: String.
		@param url: URL vers le fichier pdf.

		@type  log: Logger.
		@param log: Fichier de log.

		@type  tc: TextCleaner.
		@param tc: Nettoyeur de texte.
	"""
    # On prend l'URL de base pour désigner l'origine du fichier à laquelle on enlève le "www."
    origine = re.sub(r'^www\.', '', urllib.request.urlparse(url).netloc)
    url_split = url.split('/')

    # Extraction du nom du document
    if url_split[-1] == '':
        nom_document = norm_string(url_split[-2])
    else:
        nom_document = norm_string(url_split[-1])

    src_file_path = "{0}{1}/Documents_SRC/[{2}]{3}".format(CHEMIN_RESULTATS, ville, origine, nom_document)

    if not os.path.isfile(src_file_path):
        req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
        response = urllib.request.urlopen(req)
        read_buffer = response.read()
        mime_type = magic.from_buffer(read_buffer, mime=True)

        source = {
            'file_link': src_file_path,
            'complete_url': url,
            'base_url': origine,
            'open_access': False,
        }

        document = {
            'name': nom_document,
            'mime_type': mime_type,
            'source': source,
            'manually_validated': False,
        }

        # Écriture du fichier téléchargé
        with open(src_file_path, "wb") as fichier:
            fichier.write(read_buffer)
            logger.info("Document sauvegardé.")

        # On détecte le type du document source pour utiliser la conversion adaptée
        # Si le document est un PDF
        if mime_type == 'application/pdf':
            try:
                # Conversion en texte brut
                raw_text = convert_pdf_to_txt(src_file_path)
                logger.info("Document PDF converti en texte brut.")
                p_d = get_pdf_info(src_file_path)
                document['text'] = tc.clean(raw_text)
                document['post_date'] = p_d
                logger.info("Texte nettoyé.\n")
            except:
                logger.exception("Erreur lors de la conversion en texte du pdf.\n")

        # Si le document est un HTML (ou autre fichier web)
        elif mime_type == 'text/html':
            try:
                # Conversion en texte brut
                metadata, raw_text = convert_html_to_txt(src_file_path)
                logger.info("Document web converti en texte brut.")
                document['text'] = tc.clean(raw_text)
                document.update(metadata)
                logger.info("Texte nettoyé.\n")

            except:
                logger.exception("Erreur lors de la conversion en texte de la page web.\n")

        else:
            logger.info("Autre type de document téléchargé.\n")
        
        document = enrich_mtd(document)

        return document

    else:
        logger.info("Le document existe déjà.\n")
        return None

In [44]:
# 
def run_requete(search):
    page = requests.get(f"https://www.google.com/search?q={search}")
    soup = BeautifulSoup(page.content, "html5lib")
    links = soup.findAll("a")
    url_list = []
    for link in links :
        print(link)
        print("QQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQQ")
        link_href = link.get('href')
        if "url?q=" in link_href:# and not "webcache" in link_href:
            # print (link.get('href').split("?q=")[1].split("&sa=U")[0])
            url_list.append(link.get('href').split("?q=")[1].split("&sa=U")[0])
    return url_list[:-1]


In [45]:
def recherche_web_simpl(thematic_keyword, motscles, logger, site):
    """
		Effecue la recherche.

		@type  ville: String.
		@param ville: Nom de la ville.

		@type  motscles: Liste de string.
		@param motscles: Liste des mots clés à rechercher.

		@type  logger: Logger.
		@param logger: Fichier de log.
	"""
    # Connexion à MongoDB
    client = MongoClient()
    db = client['inventaire_medo']  # Sélection de la base de données
    collection = db['agriculture_git']  # Sélection de la collection

    # Initialisation du nettoyeur de texte
    tc = TextCleaner()
    # Génération de la liste des requêtes
    liste_requetes = generation_requetes_(formatage_mots(thematic_keyword), motscles, logger, site)
    #print('>>#', liste_requetes)
    # Date du début de la requête
    # date_collecte = datetime.now().strftime("%Y-%m-%d")
    #date_collecte = datetime.now()

    for requete in liste_requetes:
        #print('###', requete)
        response = search(requete, lang='fr', stop=10) # stop=10
        #response = run_requete(requete)  # top@10 first pages
        # response = bing_search.search(requete)  # stop=10
        try:
            #liste_url = list(response)
            liste_url = set(response)
        except:
            liste_url = []
        with (open("{0}{1}/resume.txt".format(CHEMIN_RESULTATS, thematic_keyword), "a")) as res:
            logger.info("Nouvelle requête : {0}\n".format(requete))

            res.write("Requête : {0}\n".format(requete))
            res.write("Nombre de résultats affichés : {0}\n".format(len(liste_url)))
            res.write("\nListe des résultats\n")
            res.write("\n")

            for url in liste_url:
                if not url.endswith('xml.gz'):
                    logger.info("URL : {0}\n".format(url))
                    res.write("{0}\n".format(url))
                    try:
                        # On temporise chaque requête Google pour ne pas être bloqué
                        with limitation_temporelle(30):                       
                            document = sauvegarde_fichier_simpl(thematic_keyword, url, logger, tc)
                            # On s'assure que le document existe et qu'un texte y est associé
                            if document and 'text' in document:
                                
                                document['source']['type'] = 'web_request'
                                document['source']['raw_request'] = requete
                                #document['collection_date'] = date_collecte
                                document['thematic'] = thematic_keyword
                                logger.info("Document inséré dans l'inventaire.\n")

                    except Exception as e:
                        print(e)
                        logger.info("Erreur pour l'URL : {0}\n".format(url))
                        res.write("Erreur pour l'URL : {0}\n".format(url))

            logger.info("***********************************************************************\n")
            res.write("***********************************************************************\n\n")

        with open("{0}{1}/.sauvegarde.txt".format(CHEMIN_RESULTATS, thematic_keyword), "a") as f:
            f.write("{0}\n".format(requete))

        pause(logger)
        
        #score_cps_and_mrgd_syno_terms_sim(document['text'], corpus_file_syno)
        return document

In [46]:
def recherche_web_advanced(spatial_extent,voc_concept, motscles, logger, site):
    """
		Effecue la recherche.

		@type  ville: String.
		@param ville: Nom de la ville.

		@type  motscles: Liste de string.
		@param motscles: Liste des mots clés à rechercher.

		@type  logger: Logger.
		@param logger: Fichier de log.
	"""
    # Connexion à MongoDB
    client = MongoClient()
    db = client['inventaire_medo']  # Sélection de la base de données
    collection = db['agriculture_git']  # Sélection de la collection

    # Initialisation du nettoyeur de texte
    tc = TextCleaner()

    # Liste des stopwords
    #stopwords = creer_liste_stopwords('listes_stopwords/stopwords_base.txt', 'listes_stopwords/stopwords_1000.txt')

    # Génération de la liste des requêtes
    liste_requetes = generation_requetes_(formatage_mots(spatial_extent), motscles, logger, site)
    #print('>>#', liste_requetes)
    # Date du début de la requête
    # date_collecte = datetime.now().strftime("%Y-%m-%d")
    #date_collecte = datetime.now()

    for requete in liste_requetes:
        #print('###', requete)
        response = search(requete, lang='fr', stop=10) # stop=10
        #response = run_requete(requete)  # top@10 first pages
        # response = bing_search.search(requete)  # stop=10
        try:
            #liste_url = list(response)
            liste_url = set(response)
        except:
            liste_url = []
        with (open("{0}{1}/resume.txt".format(CHEMIN_RESULTATS, spatial_extent), "a")) as res:
            logger.info("Nouvelle requête : {0}\n".format(requete))

            res.write("Requête : {0}\n".format(requete))
            res.write("Nombre de résultats affichés : {0}\n".format(len(liste_url)))
            res.write("\nListe des résultats\n")
            res.write("\n")

            for url in liste_url:
                if not url.endswith('xml.gz'):
                    logger.info("URL : {0}\n".format(url))
                    res.write("{0}\n".format(url))
                    try:
                        # On temporise chaque requête Google pour ne pas être bloqué
                        with limitation_temporelle(30):
                            
                            document = sauvegarde_fichier_advanced(spatial_extent, url, logger, tc)

                            # On s'assure que le document existe et qu'un texte y est associé
                            if document and 'text' in document:
                                compute_best_doc(voc_concept, document)
                                document['source']['type'] = 'web_request'
                                document['source']['raw_request'] = requete
                                #document['collection_date'] = date_collecte
                                document['spatial_extent'] = spatial_extent
                                # document['textual_genre'] = genre
                                document_id = insertion_document(document, collection)
                                # logger.info("Document classé comme \"{}\".".format(genre))
                                logger.info("Document inséré dans l'inventaire.\n")

                                # else:
                                # os.remove(document['source']['file_link'])
                                # logger.info("Document non pertinent. Fichier source supprimé.\n")

                    except Exception as e:
                        print(e)
                        logger.info("Erreur pour l'URL : {0}\n".format(url))
                        res.write("Erreur pour l'URL : {0}\n".format(url))

            logger.info("***********************************************************************\n")
            res.write("***********************************************************************\n\n")

        with open("{0}{1}/.sauvegarde.txt".format(CHEMIN_RESULTATS, spatial_extent), "a") as f:
            f.write("{0}\n".format(requete))

        pause(logger)
        
        #score_cps_and_mrgd_syno_terms_sim(document['text'], corpus_file_syno)
        return document

In [3]:
# some extra to process google data before formating to BioTex inout file
def cleanhtml(raw_text, remove_punc=False, lower=False):
    """
    Replace HTML tags in a text.

    raw_html : str
        html in its raw form
    """
    clean_text = raw_text

    # Remove hmtl and url patterns
    patterns = [re.compile('<.*?>'), re.compile('\[\d\]'), re.compile('www.\S+.com')]

    for pattern in patterns:
        clean_text = re.sub(pattern, '', clean_text)

    # Special characters causing pb with Biotex
    # ['\n', '\t', 'ã', '€', "\'", "\xa0"]
    toRemove = ['\n', '\t','\"', 'ã', '€', "\xa0"]

    for char in toRemove:
        clean_text = re.sub(char, '', clean_text)

    # add whitespace after a dot
    rx = r"\.(?=\S)"
    clean_text = re.sub(rx, ". ", clean_text)

    if remove_punc:
        clean_text = re.sub('[^A-Za-z0-9]+', ' ', clean_text)

    if lower:
        clean_text = clean_text.lower()

    return clean_text.strip()


def biotex_corpus_builder(g_corpus,keywords):
    #os.chdir(files_dir)
    if not os.path.exists(os.getcwd()+'/'+str(keywords)):
        os.makedirs(os.getcwd()+'/'+str(keywords))
    root = os.getcwd() + '/' + str(keywords)
    for doc in g_corpus:
        fw = open(root + '/' + str(keywords) + '_google_corpus.txt', 'a+')
        fw.write("%s\n" % cleanhtml(g_corpus['text']))
        fw.write("\n##########END##########\n")
        fw.close()


In [5]:
def simple_scraper(thematic_keyword,keyword_file, site=''):
    logger = Log(CHEMIN_LOG, 'collecteDeDonnees_{0}'.format(datetime.today().strftime("%d-%m-%y")))
    # ville = formatage_mots(input("Sur quelle ville voulez-vous effectuer la recherche ? \n"))
    thematic_keyword = thematic_keyword.title()
    motscles = creation_dossier_resultat(CHEMIN_RESULTATS, thematic_keyword, logger,keyword_file)
    logger.info("Début de la recherche de document concernant la ville de {0}".format(thematic_keyword))
    corpus = recherche_web_simpl(thematic_keyword, motscles, logger, site)
    #recherche_web_(motscles)
    biotex_corpus_builder(corpus,thematic_keyword)
    return corpus

#main('Montpellier',keyword_file)

In [6]:
def advanced_scraper(spatial_extent,voc_concept, site=''):
    logger = Log(CHEMIN_LOG, 'collecteDeDonnees_{0}'.format(datetime.today().strftime("%d-%m-%y")))
    # ville = formatage_mots(input("Sur quelle ville voulez-vous effectuer la recherche ? \n"))
    spatial_extent = spatial_extent.title()
    motscles = creation_dossier_resultat(CHEMIN_RESULTATS, spatial_extent, logger,voc_concept)
    logger.info("Début de la recherche de document concernant la ville de {0}".format(spatial_extent))
    recherche_web_advanced(spatial_extent,voc_concept, motscles, logger, site)
    #recherche_web_(motscles)

#main('Montpellier',keyword_file)